<a href="https://colab.research.google.com/github/SathishKumar9866/FL_AV/blob/check/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone your GitHub repository
!git clone https://github.com/SathishKumar9866/FL_AV.git

fatal: destination path 'FL_AV' already exists and is not an empty directory.


In [2]:
import os

# List files in a directory
files = os.listdir('/content/FL_AV')
print(files)

['bddk100_clean.ipynb', 'yolov5su.pt', 'Server.py', 'util.py', '.gitattributes', '.git', 'trainer', 'yolov5', '.vscode', 'json_to_yolo', '.gitignore', 'README.md', 'pyproject.toml', 'data', 'bdd100k_checks.ipynb', 'config', 'main.py', 'my-project', 'FlowerClient.py', 'batch_making', 'segment', 'Research_docs', 'model', 'hubconf.py']


In [3]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available!")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")

GPU is available!


In [4]:
cd /content/FL_AV/my-project/

/content/FL_AV/my-project


In [5]:
ls

models/  my_project/  pyproject.toml  README.md  runs/  utils/  yolov5su.pt


In [6]:
!pip install flwr

In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [8]:
# Install PyTorch with CUDA support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [9]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU:", torch.cuda.get_device_name(0))

PyTorch version: 2.5.1+cu118
CUDA available: True
CUDA version: 11.8
GPU: Tesla T4


In [11]:
 pip install -U "flwr[simulation]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 MB 12.7 MB/s eta 0:00:00


In [12]:
!flwr run .

2025-02-04 03:58:25.393611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738641505.414366    4947 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738641505.421110    4947 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 03:58:25.448093: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loading project configuration... 
Success
2025-02-04 03:58:29.872659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.